## Aim - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)).(This is one of Coursera's Assignment) 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.



___

Provided with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Predictions are given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 



### DATA PREPROCESSING PHASE

In [1]:
import pandas as pd
import numpy as np
df1 = pd.read_csv("train.csv",engine = "python")
df2 = pd.read_csv("test.csv")
df1 = df1[df1["compliance"].notnull()]
y = df1[["ticket_id","compliance"]]
y.set_index("ticket_id",inplace = True)
df1 = df1.drop("compliance",axis = 1)
X = df1[["ticket_id","disposition","judgment_amount","discount_amount"]]
X_test = df2[["ticket_id","disposition","judgment_amount","discount_amount"]]
X_test.set_index("ticket_id",inplace = True)
X.set_index("ticket_id",inplace = True)
# MANUAL ENCODING
X["disposition"].replace({'Responsible by Default':1,'Responsible by Determination':2,
                         'Responsible by Admission':3,'Responsible (Fine Waived) by Deter':2},inplace = True) 
X_test["disposition"].replace({'Responsible by Default':1,'Responsible by Determination':2,
                               'Responsible by Admission':3,'Responsible (Fine Waived) by Deter':2,
                               'Responsible (Fine Waived) by Admis':3,'Responsible - Compl/Adj by Default':1,
                               'Responsible - Compl/Adj by Determi':2,'Responsible by Dismissal':0},inplace = True)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


### TRYING OUT DIFFERENT MODELS

In [2]:
#FOR WARNINGS
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action = "ignore",category = DataConversionWarning)
# for training purposes
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# Training different models to find out the best one
lr = LogisticRegression().fit(X,y)
clf1 = RandomForestClassifier().fit(X_train,y_train) #1
clf2 = DecisionTreeClassifier().fit(X_train,y_train) #2
clf3 = GaussianNB().fit(X_train,y_train)   #3
clf4 = GradientBoostingClassifier(max_depth = 10).fit(X_train,y_train)    #4

#predicting models
y_pred = lr.predict_proba(X_test)
y_pred1 = clf1.predict_proba(X_test)
y_pred2 = clf2.predict_proba(X_test)
y_pred3 = clf3.predict_proba(X_test)
y_pred4 = clf4.predict_proba(X_test)

#reshaping for model
y_pred = y_pred[:,1].reshape(-1,1)
y_pred1 = y_pred1[:,1].reshape(-1,1)
y_pred2 = y_pred2[:,1].reshape(-1,1)
y_pred3 = y_pred3[:,1].reshape(-1,1)
y_pred4 = y_pred4[:,1].reshape(-1,1)

#metrics
from sklearn.metrics import roc_curve,auc
fpr,tpr,_ = roc_curve(y_test,y_pred)
fpr1,tpr1,_ = roc_curve(y_test,y_pred1)
fpr2,tpr2,_ = roc_curve(y_test,y_pred2)
fpr3,tpr3,_ = roc_curve(y_test,y_pred3)
fpr4,tpr4,_ = roc_curve(y_test,y_pred4)

print("Logistic Regression-")
print(auc(fpr,tpr))
print("Random Forest-")
print(auc(fpr1,tpr1))
print("Decision Tree-")
print(auc(fpr2,tpr2))
print("Naive Bayes-")
print(auc(fpr3,tpr3))
print("Gradient Boosted Trees-")
print(auc(fpr4,tpr4))


Logistic Regression-
0.77247205565748
Random Forest-
0.7935113955873517
Decision Tree-
0.7931307403472341
Naive Bayes-
0.7249373344533228
Gradient Boosted Trees-
0.7931332536463634


#### RANDOM FOREST IS SELECTED FOR BEST ACCURACY(ROC CURVE)

In [3]:
# THE MAIN MODEL
def blight_model():
#     from sklearn.preprocessing import LabelEncoder
#     le = LabelEncoder()
    
    from sklearn.ensemble import RandomForestClassifier
    lr = RandomForestClassifier().fit(X,y)
    y_pred = lr.predict_proba(X_test)   # PROBABILITIES OF PAYMENTS ARE PREDICTED INSTEAD OF VALUES
    y_pred = y_pred[:,1].reshape(-1,1)
    compliance = pd.DataFrame(data = y_pred,index = X_test.index)
    compliance.columns = ["Probabilities"]
    return compliance

blight_model()
#The solution was then sent to Coursera grader server for evaluation which received around 80% Accuracy.

,Probabilities
ticket_id,
283213,0.020294
140880,0.030397
180238,0.357024
109917,0.030397
21796,0.030397
...,...
257545,0.013958
128923,0.053213
177760,0.030397
